In [ ]:
# Conceptual: git checkout -b eda-sierra-leone

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [ ]:
# --- Load Data ---
file_path = '../../data/sierraleone-bumbuna.csv' # Adjust path as per your directory structure
try:
    df = pd.read_csv(file_path)
    print("Data loaded successfully for Sierra Leone.")
except FileNotFoundError:
    print(f"Error: {file_path} not found. Please ensure the data files are in the 'data/' directory.")
    exit()

In [ ]:
# Convert 'Timestamp' to datetime and set as index
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.set_index('Timestamp')


In [ ]:
print("Initial DataFrame head:")
print(df.head())
print("\nInitial DataFrame info:")
df.info()

In [ ]:
# --- Summary Statistics & Missing-Value Report ---
print("\n--- Summary Statistics ---")
print(df.describe())

In [ ]:
print("\n--- Missing Value Report ---")
missing_values = df.isna().sum()
missing_percentage = (df.isna().sum() / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing_values, 'Missing %': missing_percentage})
print(missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False))

In [ ]:
# List columns with >5% nulls
cols_with_high_nulls = missing_df[missing_df['Missing %'] > 5].index.tolist()
if cols_with_high_nulls:
    print(f"\nColumns with more than 5% null values: {cols_with_high_nulls}")
else:
    print("\nNo columns with more than 5% null values above 5%.")

In [ ]:
# --- Outlier Detection & Basic Cleaning ---
zscore_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

for col in zscore_cols:
    if col in df.columns:
        if df[col].dtype not in ['float64', 'int64']:
            print(f"Warning: Column '{col}' is not numeric. Skipping Z-score calculation.")
            continue
        if df[col].isnull().all():
            print(f"Warning: Column '{col}' contains only NaN values. Skipping Z-score calculation.")
            continue
        
        df[f'{col}_zscore'] = np.abs(zscore(df[col].dropna()))
        outliers_count = df[df[f'{col}_zscore'] > 3].shape[0]
        print(f"Outliers (Z > 3) in {col}: {outliers_count} rows")
    else:
        print(f"Warning: Column '{col}' not found for Z-score calculation.")

df_clean = df.copy()

for col in df_clean.columns:
    if df_clean[col].dtype in ['float64', 'int64']:
        if df_clean[col].isna().any():
            median_val = df_clean[col].median()
            df_clean[col].fillna(median_val, inplace=True)
    elif df_clean[col].isna().any():
        pass # Handle non-numeric NaNs if necessary for critical columns

df_clean = df_clean.drop(columns=[col for col in df_clean.columns if '_zscore' in col], errors='ignore')

initial_rows = df_clean.shape[0]
df_clean.dropna(subset=['GHI', 'DNI', 'DHI', 'Tamb'], inplace=True)
rows_after_drop = df_clean.shape[0]
if initial_rows != rows_after_drop:
    print(f"Dropped {initial_rows - rows_after_drop} rows due to NaN values in critical columns.")

print(f"\nShape after cleaning: {df_clean.shape}")
print("Cleaned DataFrame head:")
print(df_clean.head())
print("\nCleaned DataFrame info:")
df_clean.info()

In [ ]:
# --- Export Cleaned DataFrame ---
output_dir = '../../data/'
output_path = f'{output_dir}sierra_leone_clean.csv'
df_clean.to_csv(output_path, index=True)
print(f"\nCleaned data exported to: {output_path}")

In [ ]:
# --- Time Series Analysis ---
plt.figure(figsize=(15, 12))

In [ ]:
plt.subplot(4, 1, 1)
plt.plot(df_clean.index, df_clean['GHI'])
plt.title('GHI over Time (Sierra Leone)')
plt.ylabel('GHI ($W/m^2$)')
plt.grid(True)


In [ ]:
plt.subplot(4, 1, 2)
plt.plot(df_clean.index, df_clean['DNI'])
plt.title('DNI over Time (Sierra Leone)')
plt.ylabel('DNI ($W/m^2$)')
plt.grid(True)

In [ ]:
plt.subplot(4, 1, 3)
plt.plot(df_clean.index, df_clean['DHI'])
plt.title('DHI over Time (Sierra Leone)')
plt.ylabel('DHI ($W/m^2$)')
plt.grid(True)

In [ ]:
plt.subplot(4, 1, 4)
plt.plot(df_clean.index, df_clean['Tamb'])
plt.title('Ambient Temperature over Time (Sierra Leone)')
plt.xlabel('Timestamp')
plt.ylabel('Tamb (°C)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df_clean['Month'] = df_clean.index.month
plt.figure(figsize=(10, 6))
sns.boxplot(x='Month', y='GHI', data=df_clean)
plt.title('GHI Distribution by Month (Sierra Leone)')
plt.xlabel('Month')
plt.ylabel('GHI ($W/m^2$)')
plt.grid(True)
plt.show()
df_clean.drop(columns=['Month'], inplace=True)

In [ ]:
# --- Cleaning Impact ---
if 'Cleaning' in df_clean.columns and df_clean['Cleaning'].nunique() > 1:
    df_clean['Cleaning'] = df_clean['Cleaning'].astype(int)
    avg_mod_by_cleaning = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()
    print("\nAverage ModA & ModB Pre/Post-Clean:")
    print(avg_mod_by_cleaning)

    avg_mod_by_cleaning.plot(kind='bar', figsize=(8, 5))
    plt.title('Average ModA & ModB by Cleaning Event (0=No Clean, 1=Clean) (Sierra Leone)')
    plt.ylabel('Average Irradiance ($W/m^2$)')
    plt.xticks(ticks=[0, 1], labels=['No Cleaning', 'Cleaning'])
    plt.grid(axis='y')
    plt.show()
else:
    print("\n'Cleaning' column not found or does not have enough variation for impact analysis.")

In [ ]:
# --- Correlation & Relationship Analysis ---
numeric_cols = df_clean.select_dtypes(include=np.number).columns.tolist()
correlation_matrix = df_clean[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap (Sierra Leone)')
plt.show()

In [ ]:
plt.figure(figsize=(18, 5))
if 'WS' in df_clean.columns and 'GHI' in df_clean.columns:
    plt.subplot(1, 3, 1)
    sns.scatterplot(x='WS', y='GHI', data=df_clean, alpha=0.6)
    plt.title('Wind Speed vs. GHI (Sierra Leone)')
    plt.xlabel('WS (m/s)')
    plt.ylabel('GHI ($W/m^2$)')

In [ ]:
if 'WSgust' in df_clean.columns and 'GHI' in df_clean.columns:
    plt.subplot(1, 3, 2)
    sns.scatterplot(x='WSgust', y='GHI', data=df_clean, alpha=0.6)
    plt.title('Wind Gust Speed vs. GHI (Sierra Leone)')
    plt.xlabel('WSgust (m/s)')
    plt.ylabel('GHI ($W/m^2$)')

In [ ]:
if 'WD' in df_clean.columns and 'GHI' in df_clean.columns:
    plt.subplot(1, 3, 3)
    sns.scatterplot(x='WD', y='GHI', data=df_clean, alpha=0.6)
    plt.title('Wind Direction vs. GHI (Sierra Leone)')
    plt.xlabel('WD (°N (to east))')
    plt.ylabel('GHI ($W/m^2$)')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
if 'RH' in df_clean.columns and 'Tamb' in df_clean.columns:
    plt.subplot(1, 2, 1)
    sns.scatterplot(x='RH', y='Tamb', data=df_clean, alpha=0.6)
    plt.title('Relative Humidity vs. Ambient Temperature (Sierra Leone)')
    plt.xlabel('RH (%)')
    plt.ylabel('Tamb (°C)')

if 'RH' in df_clean.columns and 'GHI' in df_clean.columns:
    plt.subplot(1, 2, 2)
    sns.scatterplot(x='RH', y='GHI', data=df_clean, alpha=0.6)
    plt.title('Relative Humidity vs. GHI (Sierra Leone)')
    plt.xlabel('RH (%)')
    plt.ylabel('GHI ($W/m^2$)')
plt.tight_layout()
plt.show()

In [ ]:
# --- Wind & Distribution Analysis ---
plt.figure(figsize=(12, 6))

if 'WS' in df_clean.columns:
    plt.subplot(1, 2, 1)
    sns.histplot(df_clean['WS'].dropna(), bins=30, kde=True)
    plt.title('Distribution of Wind Speed (Sierra Leone)')
    plt.xlabel('WS (m/s)')
    plt.ylabel('Frequency')

if 'WD' in df_clean.columns:
    plt.subplot(1, 2, 2)
    sns.histplot(df_clean['WD'].dropna(), bins=30, kde=True)
    plt.title('Distribution of Wind Direction (Sierra Leone)')
    plt.xlabel('WD (°N (to east))')
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
if 'GHI' in df_clean.columns:
    plt.subplot(1, 2, 1)
    sns.histplot(df_clean['GHI'].dropna(), bins=30, kde=True)
    plt.title('Distribution of GHI (Sierra Leone)')
    plt.xlabel('GHI ($W/m^2$)')
    plt.ylabel('Frequency')

if 'WS' in df_clean.columns:
    plt.subplot(1, 2, 2)
    sns.histplot(df_clean['WS'].dropna(), bins=30, kde=True)
    plt.title('Distribution of Wind Speed (Sierra Leone)')
    plt.xlabel('WS (m/s)')
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# --- Temperature Analysis ---
if 'RH' in df_clean.columns and 'Tamb' in df_clean.columns and 'GHI' in df_clean.columns:
    plt.figure(figsize=(10, 7))
    sns.scatterplot(x='RH', y='Tamb', hue='GHI', size='GHI', sizes=(20, 400),
                    data=df_clean, alpha=0.6, palette='viridis')
    plt.title('RH vs. Tamb (Colored by GHI) (Sierra Leone)')
    plt.xlabel('Relative Humidity (%)')
    plt.ylabel('Ambient Temperature (°C)')
    plt.legend(title='GHI ($W/m^2$)', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("\nSkipping Temperature Analysis: Missing one or more of 'RH', 'Tamb', 'GHI' columns.")

In [ ]:
# --- Bubble Chart ---
if 'Tamb' in df_clean.columns and 'GHI' in df_clean.columns and 'RH' in df_clean.columns and 'BP' in df_clean.columns:
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x='Tamb', y='GHI', size='RH', hue='RH', sizes=(50, 1000),
                    alpha=0.6, palette='viridis', data=df_clean)
    plt.title('GHI vs. Ambient Temperature (Bubble Size by Relative Humidity) (Sierra Leone)')
    plt.xlabel('Ambient Temperature (°C)')
    plt.ylabel('GHI ($W/m^2$)')
    plt.legend(title='Relative Humidity (%)', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
    sns.scatterplot(x='Tamb', y='GHI', size='BP', hue='BP', sizes=(50, 1000),
                    alpha=0.6, palette='magma', data=df_clean)
    plt.title('GHI vs. Ambient Temperature (Bubble Size by Barometric Pressure) (Sierra Leone)')
    plt.xlabel('Ambient Temperature (°C)')
    plt.ylabel('GHI ($W/m^2$)')
    plt.legend(title='Barometric Pressure (hPa)', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("\nSkipping Bubble Chart: Missing one or more of 'Tamb', 'GHI', 'RH', 'BP' columns.")